In [6]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable, Tuple

class GeneralSDE:
    def __init__(self, 
                 drift: Callable[[float, float], float], 
                 diffusion: Callable[[float, float], float], 
                 dt: float = 0.01):
        """
        Initialize the general SDE model.

        drift: Drift function a(t, X_t).
        diffusion: Diffusion function b(t, X_t).
        dt: Time step for the discretization.
        """
        self.drift = drift
        self.diffusion = diffusion
        self.dt = dt

class MonteCarloSimulator:
    def __init__(self, model: GeneralSDE):
        """
        Initialize the Monte Carlo simulator with a specific SDE model.

        model: Instance of GeneralSDE.
        """
        self.model = model

    def simulate(self, X0: float, T: float, num_paths: int) -> np.ndarray:
        """
        Simulate paths using the Euler-Maruyama scheme.

        X0: Initial value of the process.
        T: Total time.
        num_paths: Number of paths to simulate.
        return: Simulated paths.
        """
        N = int(T / self.model.dt)  # Number of time steps
        paths = np.zeros((N + 1, num_paths))  # Initialize paths array
        paths[0] = X0  # Set initial value
        delta_t = self.model.dt
        
        for path in range(num_paths):
            W = np.zeros(N + 1)  # Initialize Wiener process
            N_draws = np.random.normal(0, 1, N)  # Normal draws for the Wiener process
            
            for n in range(N):
                W[n+1] = W[n] + np.sqrt(delta_t) * N_draws[n]  # Update Wiener process
                drift_term = self.model.drift(n * delta_t, paths[n, path])
                diffusion_term = self.model.diffusion(n * delta_t, paths[n, path])
                paths[n+1, path] = (paths[n, path] 
                                    + drift_term * delta_t 
                                    + diffusion_term * (W[n+1] - W[n]))
        return paths

def plot_simulated_paths(paths: np.ndarray, title: str = 'Simulated Paths'):
    """
    Plot the simulated paths.

    paths: Simulated paths.
    title: Plot title.
    """
    plt.figure(figsize=(10, 6))
    plt.plot(paths)
    plt.title(title)
    plt.xlabel('Time Steps')
    plt.ylabel('Value')
    plt.show()

def gbm_drift(t: float, S: float, mu: float) -> float:
    """
    Geometric Brownian Motion drift term.

    t: Time variable.
    S: Current value of the process.
    mu: Drift coefficient.
    return Drift term.
    """
    return mu * S

def gbm_diffusion(t: float, S: float, sigma: float) -> float:
    """
    Geometric Brownian Motion diffusion term.

    t: Time variable.
    S: Current value of the process.
    sigma: Volatility coefficient.
    return: Diffusion term.
    """
    return sigma * S

# Unit tests
def test_gbm_drift():
    assert gbm_drift(0, 100, 0.05) == 5.0

def test_gbm_diffusion():
    assert gbm_diffusion(0, 100, 0.02) == 2.0

def test_simulation_shapes():
    S0 = 100
    T = 1.0
    dt = 0.01
    num_paths = 100
    drift = lambda t, S: gbm_drift(t, S, 0.05)
    diffusion = lambda t, S: gbm_diffusion(t, S, 0.02)
    gbm_model = GeneralSDE(drift=drift, diffusion=diffusion, dt=dt)
    simulator = MonteCarloSimulator(model=gbm_model)
    paths = simulator.simulate(S0, T, num_paths)
    assert paths.shape == (int(T/dt) + 1, num_paths)
    
def run_simulation():
    """
    Run the simulation for Geometric Brownian Motion.
    """
    # Parameters
    S0 = 100   # Initial stock price
    mu = 0.05  # Drift term
    sigma = 0.2  # Volatility term
    T = 1.0    # Total time
    dt = 0.01  # Time step
    num_paths = 10  # Number of paths to simulate

    # Create drift and diffusion functions
    drift = lambda t, S: gbm_drift(t, S, mu)
    diffusion = lambda t, S: gbm_diffusion(t, S, sigma)

    # Initialize the SDE model
    gbm_model = GeneralSDE(drift=drift, diffusion=diffusion, dt=dt)

    # Initialize the simulator
    simulator = MonteCarloSimulator(model=gbm_model)

    # Simulate the paths
    paths = simulator.simulate(S0, T, num_paths)

    # Plot the simulated paths
    plot_simulated_paths(paths, title='Simulated Paths of Geometric Brownian Motion')

All tests passed.


In [7]:
if __name__ == "__main__":
    # Run the simulation
    run_simulation()
    
    # Run unit tests
    test_gbm_drift()
    test_gbm_diffusion()
    test_simulation_shapes()
    print("All tests passed.")